In [1]:
import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np
import random

from tqdm import tqdm

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *


/root/miniconda3/envs/r3/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-12 11:11:28.690734: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-12 11:11:32.752662: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda/lib64
2024-11-12 11:11:32.752841: W tensorflow/compiler/xla/stream_executor/platf

In [2]:
seed = 6
torch.manual_seed(seed)  # 为CPU设置随机种子
torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)  # 为所有GPU设置随机种子
np.random.seed(seed)
random.seed(seed)

sparse_features = ['C' + str(i) for i in range(1, 27)]   #C代表类别特征 class
dense_features =  ['I' + str(i) for i in range(1, 14)]   #I代表数值特征 int
col_names = ['label'] + dense_features + sparse_features
test_col_names = dense_features + sparse_features

train_data_fraction = 0.5
train_data_chunks = pd.read_csv('./oridata/train.txt', names=col_names, sep='\t', chunksize=30000)
data = pd.concat(chunk.sample(frac=train_data_fraction, random_state=42) for chunk in tqdm(train_data_chunks, desc="Loading train data"))

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']

# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])


Loading train data: 1529it [05:26,  4.68it/s]


In [3]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].max() + 1, embedding_dim=4)
                          for feat in sparse_features] + [DenseFeat(feat, 1, )
                                                          for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(
    linear_feature_columns + dnn_feature_columns)

# 3.generate input data for model

train, test = train_test_split(data, test_size=0.2, random_state=2020)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}


In [4]:
 # 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'
else:
    print("cpu")
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                task='binary',
#                l2_reg_embedding=1e-5, device=device)
model = xDeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
               task='binary',
               l2_reg_embedding=1e-5, device=device)

model.compile("adagrad", "binary_crossentropy",
              metrics=["binary_crossentropy", "auc"], )

history = model.fit(train_model_input, train[target].values, batch_size=4096, epochs=5, verbose=1,
                    validation_split=0.2)
pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

cuda ready...
cuda:0
Train on 14668996 samples, validate on 3667250 samples, 3582 steps per epoch


3582it [06:41,  8.92it/s]


Epoch 1/5
434s - loss:  0.4677 - binary_crossentropy:  0.4677 - auc:  0.7803 - val_binary_crossentropy:  0.4611 - val_auc:  0.7884


3582it [06:36,  9.03it/s]


Epoch 2/5
428s - loss:  0.3817 - binary_crossentropy:  0.3817 - auc:  0.8632 - val_binary_crossentropy:  0.4878 - val_auc:  0.7660


...

Epoch 3/5
429s - loss:  0.3379 - binary_crossentropy:  0.3379 - auc:  0.8921 - val_binary_crossentropy:  0.5074 - val_auc:  0.7593


3582it [06:37,  9.00it/s]


Epoch 4/5
430s - loss:  0.3227 - binary_crossentropy:  0.3226 - auc:  0.9012 - val_binary_crossentropy:  0.5213 - val_auc:  0.7547


3582it [06:49,  8.74it/s]


Epoch 5/5
443s - loss:  0.3137 - binary_crossentropy:  0.3136 - auc:  0.9065 - val_binary_crossentropy:  0.5332 - val_auc:  0.7516

test LogLoss 0.5329
test AUC 0.752
